In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [ ]:
!pip install  --upgrade accelerate 
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

In [ ]:
from transformers import pipeline,set_seed
from matplotlib import pyplot as plt 
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch
nltk.download("punkt")

In [ ]:
device = torch.device("cpu")
print("Using device:", device)

In [ ]:
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [ ]:
!wget https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
!unzip summarizer-data.zip

In [ ]:
import pandas as pd
from datasets import load_from_disk

dataset = pd.read_csv("small_samsum-test.csv")
dataset

In [ ]:
dataset.columns

In [ ]:

def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }


In [ ]:
dataset_samsum_pt = dataset.map(convert_examples_to_features, batched = True)

In [ ]:
dataset_samsum_pt["train"]

In [ ]:
#Start training

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)


In [ ]:

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)


from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)



trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"])



In [ ]:
trainer.train()

In [1]:
# Import necessary libraries
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, DataCollatorForSeq2Seq, TrainingArguments, Trainer

# Load the CSV file into a Pandas DataFrame
# Note: Make sure the CSV file is in the correct directory or provide the absolute path
dataset = pd.read_csv("small_samsum-test.csv")

# Convert the DataFrame into a Hugging Face Dataset
# Comment: Since the 'datasets' library expects a Dataset object, convert the DataFrame to it
hf_dataset = Dataset.from_pandas(dataset)

# Load the Pegasus tokenizer
# Comment: Load the tokenizer to preprocess the text data
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-cnn_dailymail")

# Function to convert examples to features for the model
def convert_examples_to_features(example_batch):
    # Encode the dialogue with truncation to fit model input limits
    input_encodings = tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)
    
    # Set the tokenizer to use the target format and encode the summary
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length=128, truncation=True)
    
    # Return the formatted encodings
    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

# Apply the conversion function to the dataset
# Comment: Use `map` from the Hugging Face Dataset library, not Pandas
dataset_samsum_pt = hf_dataset.map(convert_examples_to_features, batched=True)

# Load the Pegasus model for conditional generation
model_pegasus = PegasusForConditionalGeneration.from_pretrained("google/pegasus-cnn_dailymail")

# Set up a data collator for sequence-to-sequence tasks
# Comment: Data collator handles padding dynamically
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

# Configure the training arguments
trainer_args = TrainingArguments(
    output_dir='pegasus-samsum',
    num_train_epochs=1,
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=500,
    save_steps=1_000_000,  # Use integer notation for clarity
    gradient_accumulation_steps=16
)

# Initialize the Trainer object
# Comment: Define the Trainer with model, arguments, datasets, tokenizer, and data collator
trainer = Trainer(
    model=model_pegasus,
    args=trainer_args,
    tokenizer=tokenizer,
    data_collator=seq2seq_data_collator,
    train_dataset=dataset_samsum_pt,  # Update with the correct training dataset
    eval_dataset=dataset_samsum_pt  # Update with the correct evaluation dataset
)

# Start the training process
trainer.train()


/home/ayush/Documents/AI/Machine_Learning/Projects/Paragraph-Summerizer/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map:   0%|          | 0/200 [00:00<?, ? examples/s]/home/ayush/Documents/AI/Machine_Learning/Projects/Paragraph-Summerizer/env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 200/200 [00:00<00:00, 706.47 examples/s]
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasu

: 